In [2]:
from utils import *
from copy import deepcopy

ref_length = int(30)
H12, false_variant_locs = create_reference_hap(ref_length=ref_length, 
                                               false_variance = 0.1)
ref_H = deepcopy(H12)
samples, st_en = generate_fragments(ref_H,
                                    read_length=5, 
                                    std_read_length=1, 
                                    coverage=6,)


hap_samples = deepcopy(samples)
start_end = deepcopy(st_en)
S1, ST_EN = cluster_fragments(hap_samples, st_en)



[ 3  9 15]
0 0 1 0 1 1 0 0 0 1 0 1 1 0 1 0 0 1 0 0 0 1 0 0 1 1 0 1 1 1
1 1 0 0 0 0 1 1 1 1 1 0 0 1 0 0 1 0 1 1 1 0 1 1 0 0 1 0 0 0




In [12]:
er_hap_samples, sts_ens, frag_file_contents = simulate_haplotypes_errors(hap_samples = S1, 
                                                                         reads_st_en = ST_EN,
                                                                         false_variant_locs = false_variant_locs)

In [31]:
generate_matrix_for_visualization(ref_H, false_variant_locs, # er_hap_samples, sts_ens)
                                  S1[2+1:4+1], ST_EN[2+1:4+1])


False variant locations are [3, 9, 15]
reference haplotype values for the variants are:


0 0 1 0 1 1 0 0 0 1 0 1 1 0 1 0 0 1 0 0 0 1 0 0 1 1 0 1 1 1
1 1 0 0 0 0 1 1 1 1 1 0 0 1 0 0 1 0 1 1 1 0 1 1 0 0 1 0 0 0


- 0 1 1 1 1 - - - - - - - - - - - - - - - - - - - - - - - -
- - 0 1 0 0 - - - - - - - - - - - - - - - - - - - - - - - -


In [62]:
def get_likelihood_heterozygous_genotype(reads, st_en, index, qual):
    """
    Return the likelihood of the site at index being a heterozygous genotype, for the reads 
    - coming from same Haplotype,
    - coming from the different Haplotype.
    """
    import ipdb;ipdb.set_trace()
    assert len(reads) == 2 and len(st_en)==2
    assert index>=max(st_en[0][0], st_en[1][0]) and index<=min(st_en[0][1], st_en[1][1])
    index0, index1 = index - st_en[0][0], index - st_en[1][0]
    if reads[0][index0] == reads[1][index1]:
        # Same value at the site
        likelihood_same = ((1-qual)*(1-qual))/2 + (qual*qual)/2
        likelihood_diff = qual * (1 - qual)
    else:
        # Different values at the given site
        likelihood_same = qual * (1 - qual)
        likelihood_diff = ((1-qual)*(1-qual))/2 + (qual*qual)/2
    return likelihood_same, likelihood_diff
        

In [63]:
     
def calculate_likelihood_of_heterozygous_site(ref_H, false_variant_locs, reads, st_en, index, qual):
    """
    Given the reads what is the likelihood of allele lying at index being 
    a heterozygous genotype.
    param qual: the qual of the reads, currently it is a constant.maketrans()
    # TODO: Make qual an array which stores the qual of each of the index.
    """
    import ipdb;ipdb.set_trace()
    reads, st_en = cluster_fragments(reads, st_en)
    overlapping_reads, overlapping_st_en = get_overlapping_fragments_for_variants_sites(reads, st_en, index)    
    #generate_matrix_for_visualization(ref_H, false_variant_locs, overlapping_reads, overlapping_st_en)
    num_frags = len(overlapping_reads)
    
    # Let the first bit for H1 be 0 and first bit of H2 = 1( without loss of generality) 
    # Likelihood for the reads lying in H1, H2 given i fragments are stored in the ith index of the
    # array likelihood_per_reads
    
    ipdb.set_trace()
    frag_0 = (overlapping_reads[0], overlapping_st_en[0])
    likelihood_per_reads = [(1-qual, qual) if overlapping_reads[0][0] == 0 else (qual, 1-qual)]
    
    for i in range(1, num_frags):
        frag_1 = (overlapping_reads[i], overlapping_st_en[i])
        r, s_e = [frag_0[0], frag_1[0]],  [frag_0[1], frag_1[1]]
        
        prob_same, prob_diff = get_probability_fragments_from_same_fragment(r, s_e, index, qual)
             
        same_lik, diff_lik = get_likelihood_heterozygous_genotype(r, s_e, index, qual)
        
        l1_prev, l2_prev = likelihood_per_reads[-1]
        
        # l1 is the likelihood that the xi fragment is sampled from the H1
        # l2 is the likelihood that the xi fragment is sampled from the H2.         
        l1 = l1_prev*prob_same*same_lik + l2_prev*prob_diff*diff_lik
        l2 = l1_prev*prob_diff*diff_lik + l2_prev*prob_same*same_lik
        
        likelihood_per_reads.append((l1, l2))       
        
        frag_0 = frag_1
    # Finally my likelihood per_reads would contain,
    # the likelihood of the site at index to be heterozygous
    heteroz_likelihood = likelihood_per_reads[-1][0] + likelihood_per_reads[-1][1]
    return  heteroz_likelihood
        

In [65]:
calculate_likelihood_of_heterozygous_site(ref_H, false_variant_locs, samples, st_en, 22, 0.02)

> <ipython-input-63-74a30aed7d37>(9)calculate_likelihood_of_heterozygous_site()
      8     import ipdb;ipdb.set_trace()
----> 9     reads, st_en = cluster_fragments(reads, st_en)
     10     overlapping_reads, overlapping_st_en = get_overlapping_fragments_for_variants_sites(reads, st_en, index)

> <ipython-input-63-74a30aed7d37>(19)calculate_likelihood_of_heterozygous_site()
     18     ipdb.set_trace()
---> 19     frag_0 = (overlapping_reads[0], overlapping_st_en[0])
     20     likelihood_per_reads = [(1-qual, qual) if overlapping_reads[0][0] == 0 else (qual, 1-qual)]

> <ipython-input-62-1d6711cc3c03>(8)get_likelihood_heterozygous_genotype()
      7     import ipdb;ipdb.set_trace()
----> 8     assert len(reads) == 2 and len(st_en)==2
      9     assert index>=max(st_en[0][0], st_en[1][0]) and index<=min(st_en[0][1], st_en[1][1])

> <ipython-input-62-1d6711cc3c03>(9)get_likelihood_heterozygous_genotype()
      8     assert len(reads) == 2 and len(st_en)==2
----> 9     assert index>=

BdbQuit: 

In [47]:

over_S, over_st_en =  get_overlapping_fragments_for_variants_sites(S1, ST_EN, 22)
generate_matrix_for_visualization(ref_H, false_variant_locs, # er_hap_samples, sts_ens)
                                  over_S, over_st_en)

False variant locations are [3, 9, 15]
reference haplotype values for the variants are:


0 0 1 0 1 1 0 0 0 1 0 1 1 0 1 0 0 1 0 0 0 1 0 0 1 1 0 1 1 1
1 1 0 0 0 0 1 1 1 1 1 0 0 1 0 0 1 0 1 1 1 0 1 1 0 0 1 0 0 0


- - - - - - - - - - - - - - - - 0 1 0 0 0 1 - - - - - - - -
- - - - - - - - - - - - - - - - 0 1 0 0 0 1 - - - - - - - -
- - - - - - - - - - - - - - - - - 1 0 0 0 1 0 - - - - - - -
- - - - - - - - - - - - - - - - - - 1 1 1 0 1 1 - - - - - -
- - - - - - - - - - - - - - - - - - - 1 1 0 1 - - - - - - -
- - - - - - - - - - - - - - - - - - - 0 0 1 0 0 1 - - - - -
- - - - - - - - - - - - - - - - - - - - 1 0 1 1 - - - - - -
- - - - - - - - - - - - - - - - - - - - 0 1 0 0 - - - - - -
- - - - - - - - - - - - - - - - - - - - 1 0 1 1 0 - - - - -
- - - - - - - - - - - - - - - - - - - - - 0 1 1 0 0 1 - - -
- - - - - - - - - - - - - - - - - - - - - - 0 0 1 1 0 - - -


In [33]:
print(get_probability_fragments_from_same_fragment(S1[2:2+2], ST_EN[2:2+2], 3, 0.02))
# for i in range(len(S1)-2):
#     print(get_probability_fragments_from_same_fragment(S1[i:i+2], ST_EN[i:i+2], 3, 0.02)

(0.9411919999999999, 8.000000000000001e-06)


False variant locations are [0, 3, 18]
reference haplotype values for the variants are:


1 1 1 1 1 0 0 0 1 1 1 1 0 1 1 0 1 0 0 1 1 1 1 1 1 1 1 0 1 0
1 0 0 1 0 1 1 1 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 1 0 1


1 1 1 1 1 0 - - - - - - - - - - - - - - - - - - - - - - - -
1 1 1 1 1 0 - - - - - - - - - - - - - - - - - - - - - - - -
- - 0 1 0 1 - - - - - - - - - - - - - - - - - - - - - - - -
- - - 1 1 0 0 0 - - - - - - - - - - - - - - - - - - - - - -
- - - - 0 1 1 1 - - - - - - - - - - - - - - - - - - - - - -
- - - - 1 0 0 0 1 - - - - - - - - - - - - - - - - - - - - -
- - - - - 0 0 0 1 1 - - - - - - - - - - - - - - - - - - - -
- - - - - - 1 1 0 0 - - - - - - - - - - - - - - - - - - - -
- - - - - - 1 1 0 0 0 0 1 - - - - - - - - - - - - - - - - -
- - - - - - - 1 0 0 0 - - - - - - - - - - - - - - - - - - -
- - - - - - - 0 1 1 1 1 - - - - - - - - - - - - - - - - - -
- - - - - - - - - 0 0 0 - - - - - - - - - - - - - - - - - -
- - - - - - - - - - 1 1 0 1 - - - - - - - - - - - - - - - -
- - - - 

Notes: We want to incorporate sequencing errors in the fragments f1 and f2 and get the confidence whether f1 and f2 belong from the same haplotype. Use some probablity to guess whether f1 and f2 belong to the same haplotype. 

Now once we have a probablity that f1 and f2 belong to the same haplotype, we want to check whether a variant is a false variant.

What is a false variant ? 
False variant is a location where the alleles are homozygous but the sequencing 
assumes / wrongly interprets the site as heterozygous. 
Hitherto, in the fragments (for these false variants) there would be a substantial amount of fragments where this site has the same allele say 0 for 
both fragments coming from both H1 and H2 (as it is not really a variant hence same allele is observed irrespective sample from H1 or H2 (0, 0) in the given position).

However for sequencing errors the errors are randomly distributed among different variants for different fragments.

Problem statement, we want to identify these false variant locations for the 
sequenced data, as these become difficult to handle while performing Hap assembly.

When we have a good coverage for a particular false variant, 
if we have fragment f1 which belongs to H1 and a fragment f2 which belongs to H2, the value for the variants are complement of each other except for the false variant. 

Now we dont know whether the fragment belongs to either H1 or H2, 